# MusicCaps to Datasette

Make something neat out of:
    
- https://raw.githubusercontent.com/tensorflow/models/master/research/audioset/yamnet/yamnet_class_map.csv
- https://gist.github.com/simonw/cd138ed5effebc0924bf174942a7b3e0

In [1]:
# Figure out how to turn a Python array into JSON
tags = "['low quality', 'sustained strings melody', 'soft female vocal', 'mellow piano melody', 'sad', 'soulful', 'ballad']"

In [97]:
import ast, json

In [26]:
ast.literal_eval(tags)

['low quality',
 'sustained strings melody',
 'soft female vocal',
 'mellow piano melody',
 'sad',
 'soulful',
 'ballad']

In [27]:
import csv

In [36]:
import httpx, io

In [31]:
audioset_csv = httpx.get("https://raw.githubusercontent.com/tensorflow/models/master/research/audioset/yamnet/yamnet_class_map.csv").text

In [175]:
# Build an array to look up audioset names by their IDs
audioset_lookup = {
    r['mid']: r['display_name']
    for r in list(csv.DictReader(io.StringIO(audioset_csv)))
}
list(audioset_lookup.items())[:3]

[('/m/09x0r', 'Speech'),
 ('/m/0ytgt', 'Child speech, kid speaking'),
 ('/m/01h8n0', 'Conversation')]

In [40]:
youtube_key = '... API key goes here ...'

In [2]:
# Fetch video data from the YouTube API
def details_for_video_ids(video_ids):
    response = httpx.get("https://www.googleapis.com/youtube/v3/videos", params={
        "part": "snippet,statistics",
        "id": ",".join(video_ids),
        "key": youtube_key,
    })
    items = []
    for raw_item in response.json()["items"]:
        item = {"id": raw_item["id"]}
        item.update(raw_item["snippet"])
        # Convert stats to integers
        item.update({k: int(v) for k, v in raw_item["statistics"].items()})
        items.append(item)
    return items

In [58]:
r = details_for_video_ids(["-FlvaZQOr2I"]); r

[{'id': '-FlvaZQOr2I',
  'publishedAt': '2014-10-25T21:20:40Z',
  'channelId': 'UCy41ACXafDUcOMIBN90ZE1g',
  'title': "[Growtopia] Why Noobs Can't Spell",
  'description': '100% Illuminati. Where is the world going into?\n-------------------------------------------------------------------------------------------------------------------------------------\nCocoaCookieP Channel►https://www.youtube.com/user/CocoaCookieP\nFesliyan Studios►https://www.youtube.com/watch?v=NWaNW-aYhVY\n\nWanna Play Growtopia? Links Below...\nPC►http://growtopiagame.com/GrowtopiaIns...\nMac►http://growtopia.org/Growtopia.dmg\niOS►http://bit.ly/1fP8dvP\nAndroid►http://bit.ly/1ciiWhv',
  'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/-FlvaZQOr2I/default.jpg',
    'width': 120,
    'height': 90},
   'medium': {'url': 'https://i.ytimg.com/vi/-FlvaZQOr2I/mqdefault.jpg',
    'width': 320,
    'height': 180},
   'high': {'url': 'https://i.ytimg.com/vi/-FlvaZQOr2I/hqdefault.jpg',
    'width': 480,
    'heigh

In [217]:
caps = list(csv.DictReader(io.StringIO(
    httpx.get(
        "https://gist.githubusercontent.com/simonw/cd138ed5effebc0924bf174942a7b3e0/raw/895bbab783aaac43143c9322b3ad4289ad0ae36d/musiccaps-public.csv"
    ).text
)))

In [64]:
ids = [cap['ytid'] for cap in caps]

In [65]:
len(ids)

5521

In [66]:
def batch(iterable, n=1):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]

In [67]:
batches = [list(b) for b in batch(ids, 50)]

In [69]:
len(batches[0]), len(batches)

(50, 111)

In [ ]:
import time
videos = []
for batch in batches:
    videos.extend(details_for_video_ids(batch))
    time.sleep(2)

In [71]:
len(videos)

5503

In [75]:
import sqlite_utils
db = sqlite_utils.Database("/tmp/musiccaps.db")

In [79]:
db["videos"].insert_all(videos, pk="id", alter=True, replace=True)

<Table videos (id, publishedAt, channelId, title, description, thumbnails, channelTitle, tags, categoryId, liveBroadcastContent, localized, viewCount, likeCount, favoriteCount, commentCount, defaultAudioLanguage, defaultLanguage)>

In [218]:
caps[0]

{'ytid': '-0Gj8-vB1q4',
 'start_s': '30',
 'end_s': '40',
 'audioset_positive_labels': '/m/0140xf,/m/02cjck,/m/04rlf',
 'aspect_list': "['low quality', 'sustained strings melody', 'soft female vocal', 'mellow piano melody', 'sad', 'soulful', 'ballad']",
 'caption': 'The low quality recording features a ballad song that contains sustained strings, mellow piano melody and soft female vocal singing over it. It sounds sad and soulful, like something you would hear at Sunday services.',
 'author_id': '4',
 'is_balanced_subset': 'False',
 'is_audioset_eval': 'True'}

In [222]:
for cap in caps:
    cap["url"] = f"https://www.youtube.com/watch?v={cap['ytid']}&start={cap['start_s']}&end={cap['end_s']}"
    cap["aspect_list"] = ast.literal_eval(cap["aspect_list"])
    for key in ("is_balanced_subset", "is_audioset_eval"):
        cap[key] = {"True": 1, "False": 0}[cap[key]]
    cap["audioset_ids"] = cap.pop("audioset_positive_labels").split(",")
    cap["audioset_names"] = [audioset_lookup.get(id, id) for id in cap["audioset_ids"]]

In [223]:
caps[0]

{'ytid': '-0Gj8-vB1q4',
 'start_s': '30',
 'end_s': '40',
 'aspect_list': ['low quality',
  'sustained strings melody',
  'soft female vocal',
  'mellow piano melody',
  'sad',
  'soulful',
  'ballad'],
 'caption': 'The low quality recording features a ballad song that contains sustained strings, mellow piano melody and soft female vocal singing over it. It sounds sad and soulful, like something you would hear at Sunday services.',
 'author_id': '4',
 'is_balanced_subset': 0,
 'is_audioset_eval': 1,
 'url': 'https://www.youtube.com/watch?v=-0Gj8-vB1q4&start=30&end=40',
 'audioset_ids': ['/m/0140xf', '/m/02cjck', '/m/04rlf'],
 'audioset_names': ['Christmas music', 'Theme music', 'Music']}

In [224]:
db["musiccaps"].drop(ignore=True)

In [225]:
db["musiccaps"].insert_all(caps, pk="ytid", column_order="ytid, url, caption, aspect_list, audioset_names, author_id".split(", "))

<Table musiccaps (ytid, url, caption, aspect_list, audioset_names, author_id, start_s, end_s, is_balanced_subset, is_audioset_eval, audioset_ids)>

In [226]:
db["musiccaps"].enable_fts(["caption"], replace=True)

<Table musiccaps (ytid, url, caption, aspect_list, audioset_names, author_id, start_s, end_s, is_balanced_subset, is_audioset_eval, audioset_ids)>

In [228]:
next(db["musiccaps"].rows)

{'ytid': '-0Gj8-vB1q4',
 'url': 'https://www.youtube.com/watch?v=-0Gj8-vB1q4&start=30&end=40',
 'caption': 'The low quality recording features a ballad song that contains sustained strings, mellow piano melody and soft female vocal singing over it. It sounds sad and soulful, like something you would hear at Sunday services.',
 'aspect_list': '["low quality", "sustained strings melody", "soft female vocal", "mellow piano melody", "sad", "soulful", "ballad"]',
 'audioset_names': '["Christmas music", "Theme music", "Music"]',
 'author_id': '4',
 'start_s': '30',
 'end_s': '40',
 'is_balanced_subset': 0,
 'is_audioset_eval': 1,
 'audioset_ids': '["/m/0140xf", "/m/02cjck", "/m/04rlf"]'}

In [235]:
sql_for_view = """select
  musiccaps.url as video,
  json_object(
    'label',
    coalesce(videos.title, 'Missing from YouTube'),
    'href',
    musiccaps.url
  ) as youtube_link,
  musiccaps.caption as musiccaps_caption,
  videos.publishedAt as youtube_published,
  videos.channelTitle as youtube_channel,
  videos.description as youtube_description,
  musiccaps.audioset_names as musiccaps_names,
  musiccaps.aspect_list as musiccaps_aspects,
  musiccaps.author_id as musiccaps_author,
  videos.id as youtube_id,
  musiccaps.rowid as musiccaps_rowid
from
  musiccaps
  left join videos on musiccaps.ytid = videos.id"""

In [236]:
db.create_view("musiccaps_details", sql_for_view, replace=True)

<Database <sqlite3.Connection object at 0x11844a200>>